Import from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#%cd /drive

Mounted at /content/drive/


In [ ]:
#pwd! #kita lg ada di mana 

'/drive'

In [ ]:
import os

base_dir = '/content/drive/MyDrive/exe_img'
!ls "/content/drive/MyDrive/exe_img"

bahan  train  validation


In [ ]:
import tensorflow as tf

In [ ]:
# Menentukan direktori

bahan_dir = os.path.join(base_dir, 'bahan')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
# Menetukan direktori isi bahan

hammer_dir = os.path.join(bahan_dir, 'hammer_curl/')
barbell_dir = os.path.join(bahan_dir, 'barbell_curl/')
chestfly_dir = os.path.join(bahan_dir, 'chest_fly_machine/')
deadlift_dir = os.path.join(bahan_dir, 'deadlift/')
pulldown_dir = os.path.join(bahan_dir, 'lat_pulldown/')
extension_dir = os.path.join(bahan_dir, 'leg_extension/')

print("Jumlah Data Train Tiap Kelas")
print("Jumlah gambar hammer : ", len(os.listdir(hammer_dir)))
print("Jumlah gambar barbel : ", len(os.listdir(barbell_dir)))
print("Jumlah gambar chest fly machine : ", len(os.listdir(chestfly_dir)))
print("Jumlah gambar deadlift : ", len(os.listdir(deadlift_dir)))
print("Jumlah gambar lat_pulldown : ", len(os.listdir(pulldown_dir)))
print("Jumlah gambar leg extension : ", len(os.listdir(extension_dir)))

Jumlah Data Train Tiap Kelas
Jumlah gambar hammer :  546
Jumlah gambar barbel :  705
Jumlah gambar chest fly machine :  527
Jumlah gambar deadlift :  530
Jumlah gambar lat_pulldown :  646
Jumlah gambar leg extension :  586


In [ ]:
# Directori isi training

hammer_dir = os.path.join(train_dir, 'hammer_curl/')
barbell_dir = os.path.join(train_dir, 'barbell_curl/')
chestfly_dir = os.path.join(train_dir, 'chest_fly_machine/')
deadlift_dir = os.path.join(train_dir, 'deadlift/')
pulldown_dir = os.path.join(train_dir, 'lat_pulldown/')
extension_dir = os.path.join(train_dir, 'leg_extension/')

# Directori isi validation
hammer_dir = os.path.join(validation_dir, 'hammer_curl/')
barbell_dir = os.path.join(validation_dir, 'barbell_curl/')
chestfly_dir = os.path.join(validation_dir, 'chest_fly_machine/')
deadlift_dir = os.path.join(validation_dir, 'deadlift/')
pulldown_dir = os.path.join(validation_dir, 'lat_pulldown/')
extension_dir = os.path.join(validation_dir, 'leg_extension/')

Menyiapkan dataset

In [ ]:
import random
from shutil import copyfile

def train_val_split(source, train, val, train_ratio):
  total_size = len(os.listdir(source))
  train_size = int(train_ratio * total_size)
  val_size = total_size - train_size

  randomized = random.sample(os.listdir(source), total_size)
  train_files = randomized[0:train_size]
  val_files = randomized[train_size:total_size]

  for i in train_files:
    i_file = str(source + 1)
    destination = train +1
    copyfile(i_file, destination)

  for i in val_files:
    i_file = source + 1
    destination = train +1
    copyfile(i_file, destination)

# Jumlah pembagian data training dan testing
train_ratio = 0.8

#Pembagian training dan Validasi
#training
source_00 = hammer_dir
source_00 = barbell_dir
source_00 = chestfly_dir 

train_00 = hammer_dir
train_00 = barbell_dir
train_00 = chestfly_dir 

val_00 = hammer_dir
val_00 = barbell_dir
val_00 = chestfly_dir

train_val_split(source_00, train_00, val_00, train_ratio)

# Validation
source_01 = deadlift_dir
source_01 = pulldown_dir
source_01 = extension_dir 

train_01 = deadlift_dir
train_01 = pulldown_dir
train_01 = extension_dir 

val_01 = deadlift_dir
val_01 = pulldown_dir
val_01 = extension_dir 

train_val_split(source_01, train_01, val_00, train_ratio)

TypeError: ignored

In [ ]:
print("Jumlah gambar barbel : ", len(os.listdir(barbell_dir)))
print("Jumlah train barbel : ", len(os.listdir(train_barbell)))
print("Jumlah validation barbel : ", len(os.listdir(barbell_dir)))

Jumlah gambar barbel :  704


NameError: ignored

Pre processing

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip = True,
    shear_range = 0.3,
    fill_mode = 'nearest',
    width_shift_range = 0.2,
    zoom_range = 0.1
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip = True,
    shear_range = 0.3,
    fill_mode = 'nearest',
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.1
)

Target

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

Found 0 images belonging to 7 classes.
Found 0 images belonging to 7 classes.


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = ()):
    if(logs.get('accuracy') > 0.50):
      print('\nAkurasi mencapai 50%')
      self.model.stop_training = True
    

callbacks = myCallback()

Model CNN

In [ ]:
from keras.layers.serialization import activation
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.3, seed=112),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5, seed=112),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 64)       0

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 7,
    epoch = 25,
    validation_data = val_generator,
    validation_steps = 1,
    verbose = 1,
    callback =[callbacks]
)

TypeError: ignored

Tingkat Akurasi

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch = range(len(acc))

plt.plot(epoch, acc, 'r', label = 'Training Accuracy')
plt.plot(epoch, val, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc = 'best')
plt.show()

Klasifikasi

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  #predicting images
  path = fn
  img = iamge.load_img(path, target_size = (150, 150))
  imgplot = plt.imgshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  clases = model.predict(images, batch_size = 100)

  print(fn)

  class_list = os.listdir(train_dir)

  for j in range(42):
    if classes[0][j] == 1. :
      print('This imagebelongsto class', class_list[j-1])
      break


In [ ]:
import os
import glob
import zipfile
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
hammer_curl = glob.glob('MyDrive/Exercise Images/hammer curl/*.*')
barbell_curl = glob.glob('MyDrive/Exercise Images/barbell biceps curl/*.*')
chest_fly_machine = glob.glob('MyDrive/Exercise Images/chest fly machine/*.*')
deadlift = glob.glob('MyDrive/Exercise Images/deadlift/*.*')
lat_pulldown = glob.glob('MyDrive/Exercise Images/lat pulldown/*.*')
leg_extension = glob.glob('MyDrive/Exercise Images/leg extension/*.*')
shoulder_press = glob.glob('MyDrive/Exercise Images/shoulder press/*.*')

In [ ]:
data = []
labels = []

In [ ]:
for i in hammer_curl:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(0)

for i in barbell_curl:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(1)

for i in chest_fly_machine:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(2)

for i in deadlift:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(3)
  
for i in lat_pulldown:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(4)

for i in leg_extension:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(5)

for i in shoulder_press:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (150,150))
    image=np.array(image)
    data.append(image)
    labels.append(6)

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils as u

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

NameError: ignored

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
X_train, X_test, ytrain, ytest = train_test_split(data, labels, test_size=0.2,
                                                random_state=42)

In [ ]:
(X, y), (X_test, y_test) = Exercise Images.load_data()

SyntaxError: ignored

In [ ]:
def create_model():
  # DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
  # USE AT LEAST 3 CONVOLUTION LAYERS

  ### START CODE HERE

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dense(1, activation="sigmoid")
  ])

  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss="binary_crossentropy",
                metrics=['accuracy']) 
    
  ### END CODE HERE

  return model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: ignored

In [ ]:
model.fit(data, labels, epochs=10, validation_data=(data, labels))

In [ ]:
test_loss, test_accuracy = model.evaluate(data, labels)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

In [ ]:
# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

NameError: ignored

In [ ]:
!wget --no-check-certificate https://www.kaggle.com/datasets/hasyimabdillah/workoutexercises-images

--2023-05-26 13:06:27--  https://www.kaggle.com/datasets/hasyimabdillah/workoutexercises-images
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
workoutexercises-images: Operation not supported

Cannot write to ‘workoutexercises-images’ (Success).


In [ ]:
import zipfile

# Unzip the archive
local_zip = './Exercise Images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

FileNotFoundError: ignored